# Descritores Locais (SIFT, ORB) e Matching

Diferente dos descritores globais (que olham a imagem toda), os **Descritores Locais** focam em pontos de interesse (keypoints). Isso permite reconhecer objetos mesmo com **oclusão parcial**, **rotação severa** ou **mudança de perspectiva**.

## 1) O Pipeline Local
1.  **Detecção:** Encontrar pontos estáveis (cantos, blobs) que aparecem em diferentes vistas da cena.
2.  **Descrição:** Criar um vetor único para a vizinhança desse ponto.
3.  **Matching:** Encontrar pares correspondentes entre duas imagens.
4.  **Transformação:** Usar os matches para alinhar imagens (Homografia) ou converter para uma representação global (Bag of Words).

> **Insight:** Uma vez extraídos os descritores, podemos descartar a imagem original. O conjunto de vetores é suficiente para o reconhecimento.

## 2) SIFT (Scale-Invariant Feature Transform)

Proposto por David Lowe (2004), é o padrão-ouro de robustez.

### Conceitos Chave:
- **Espaço de Escala (Scale Space):** Usa Diferença de Gaussianas (DoG) para achar pontos estáveis em vários níveis de zoom (invariância a escala).
- **Orientação Dominante:** Atribui uma direção principal ao ponto baseada no gradiente local (invariância a rotação).
- **O Descritor:** Uma grade 4x4 onde cada célula tem um histograma de 8 orientações. Resulta em um vetor de **128 dimensões** (4x4x8).

Vamos usar o OpenCV para detectar e descrever.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import data

# Preparando Imagens: Astronauta Original vs Rotacionada
img1 = cv2.cvtColor(data.astronaut(), cv2.COLOR_RGB2GRAY)
rows, cols = img1.shape
M = cv2.getRotationMatrix2D((cols/2, rows/2), 45, 0.8) # Rotação 45 graus + Scale 0.8x
img2 = cv2.warpAffine(img1, M, (cols, rows))

# 1. Criar SIFT (Se falhar, tente atualizar o opencv ou usar ORB)
try:
    sift = cv2.SIFT_create()
except AttributeError:
    print("SIFT não disponível. Usando ORB como fallback.")
    sift = cv2.ORB_create()

# 2. Detectar e Computar
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Visualizar Keypoints na Imagem 1
img_kp = cv2.drawKeypoints(img1, kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(8,8))
plt.imshow(img_kp)
plt.title(f"Keypoints SIFT detectados: {len(kp1)}")
plt.axis('off')
plt.show()

### Matching (Lowe's Ratio Test)
Ao comparar descritores (distância Euclidiana), muitos matches são ruído. Lowe propôs aceitar um match apenas se ele for **muito melhor** que o segundo melhor candidato.

$$ \frac{D(best)}{D(2nd\ best)} < 0.75 $$

In [ ]:
# Matcher força bruta (L2 para SIFT, Hamming para ORB binário)
bf = cv2.BFMatcher()

# KNN match (k=2) para aplicar Ratio Test
matches = bf.knnMatch(des1, des2, k=2)

good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)

# Desenhar Matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=2)

plt.figure(figsize=(12,6))
plt.imshow(img_matches)
plt.title(f"Matches filtrados (Ratio Test): {len(good_matches)}")
plt.axis('off')
plt.show()

## 3) Matching Geométrico e RANSAC

Mesmo com o Ratio Test, alguns matches errados (outliers) sobrevivem (veja linhas cruzadas acima). O **RANSAC** (Random Sample Consensus) resolve isso encontrando a transformação geométrica (Homografia) que explica a maioria dos pontos.

Ele basicamente diz: "Qual rotação/translação/warp alinha a maioria desses pontos? Ignore quem não se alinha."

In [ ]:
if len(good_matches) > 4:
    # Extrair coordenadas dos pontos
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Calcular Homografia com RANSAC
    M_homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    matchesMask = mask.ravel().tolist()

    print(f"RANSAC encontrou {sum(matchesMask)} inliers de {len(good_matches)} matches.")

    # Desenhar APENAS inliers
    draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
    img_ransac = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    plt.figure(figsize=(12,6))
    plt.imshow(img_ransac)
    plt.title("Matches Geométricos (RANSAC Inliers)")
    plt.axis('off')
    plt.show()
else:
    print("Matches insuficientes para homografia.")

## 4) SURF, FAST e ORB

- **SURF (Speeded-Up Robust Features):** Usa *Box Filters* e imagens integrais para ser mais rápido que o SIFT. O descritor tem 64 dimensões. (Muitas vezes protegido por patente/removido do OpenCV).
- **FAST:** Apenas detector de cantos ultra-rápido (compara pixel central com círculo ao redor). Não gera descritor.
- **ORB (Oriented FAST and Rotated BRIEF):** Alternativa livre de patentes ao SIFT/SURF. Usa FAST para detectar e BRIEF binário para descrever. É muito rápido, mas menos robusto a ângulo/escala extremos.


In [ ]:
# Exemplo Rápido de ORB na mesma imagem
orb = cv2.ORB_create()
kp_orb, des_orb = orb.detectAndCompute(img1, None)

print(f"Keypoints ORB: {len(kp_orb)}")
print(f"Descritor ORB (Binário): shape {des_orb.shape}") # Note que é uint8, não float

# Visualização rápida
img_orb = cv2.drawKeypoints(img1, kp_orb, None, color=(0,255,0))
plt.imshow(img_orb); plt.axis('off'); plt.title("ORB Keypoints"); plt.show()

## 5) Transformação: Bag of Visual Words (BoVW)

Como usar descritores locais para classificar a imagem inteira?
A ideia do **BoVW** é criar um "vocabulário" de texturas/padrões.

1.  Extrair SIFT/ORB de várias imagens.
2.  Agrupar vetores parecidos (K-Means) -> Os centros dos grupos são as "Palavras Visuais".
3.  Cada imagem vira um histograma: "Quantas vezes a palavra X apareceu aqui?".

In [ ]:
from sklearn.cluster import KMeans

# --- Mini-Demo BoVW ---
# 1. Coletar descritores de algumas imagens
images = [data.chelsea(), data.coffee(), data.astronaut(), data.camera()]
all_descriptors = []
img_descriptors = [] # Guardar descritores por imagem

sift_demo = cv2.SIFT_create()

for img in images:
    if img.ndim == 3: img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    kp, des = sift_demo.detectAndCompute(img, None)
    if des is not None:
        all_descriptors.append(des)
        img_descriptors.append(des)

if len(all_descriptors) > 0:
    # Empilhar tudo num "caldeirão" de features
    stack_des = np.vstack(all_descriptors)
    
    # 2. KMeans para criar Vocabulário (Dicionário) de 10 palavras
    kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
    kmeans.fit(stack_des)
    
    # 3. Criar Histogramas (BoVW) para cada imagem
    bovw_histograms = []
    for des in img_descriptors:
        # Predizer a qual "palavra" cada descritor da imagem pertence
        words = kmeans.predict(des)
        hist, _ = np.histogram(words, bins=10, range=(0, 10))
        
        # Normalizar
        hist = hist.astype(float) / (hist.sum() + 1e-6)
        bovw_histograms.append(hist)

    # Exibir Histogramas
    labels = ["Gato", "Café", "Astronauta", "Câmera"]
    fig, ax = plt.subplots(1, 4, figsize=(16, 3))
    for a, h, l in zip(ax, bovw_histograms, labels):
        a.bar(range(10), h)
        a.set_title(l)
        a.set_ylim(0, 0.5)
    plt.suptitle("Histogramas Bag of Visual Words (Assinatura Global)")
    plt.show()
else:
    print("Nenhum descritor encontrado para demo BoVW.")

## 6) Exercícios
1.  **SIFT vs ORB:** Tente rodar o código de matching usando `ORB_create()` e `matches = bf.match(...)` (ORB usa distância de Hamming, não L2, e não costuma usar ratio test da mesma forma, mas sim CrossCheck). Compare a qualidade no caso da rotação.
2.  **RANSAC:** Altere o limiar (threshold) do `findHomography` de 5.0 para 1.0 ou 20.0. Como isso afeta o número de inliers?
3.  **Vocabulário:** No demo BoVW, se aumentarmos `n_clusters` para 100, os histogramas ficam mais esparsos (mais zeros) ou mais cheios? Teste a intuição.